여기에 블렌딩 관련 작업

In [11]:
# from desktop w/ gpu
import tensorflow as tf
import numpy as np
import cv2
import time
import os
from enum import Enum
import keras_cv
import keras
import matplotlib.pyplot as plt
# from utils import *
from multipledispatch import dispatch

print(keras_cv.__version__)
print(keras.__version__)

print(tf.config.list_physical_devices('GPU'))
print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())
print(tf.__version__)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices() )
print(cv2.getBuildInformation())

%load_ext autoreload
%autoreload 2



0.9.0
2.15.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
4.12.0-dev
1
2.15.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11224742498720556007
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10177478656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5321042600615940873
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]

General configuration for OpenCV 4.12.0-dev =====================================
  Version control:               4.11.0-356-g325e59bd4c

  Extra modules:
    Location (extra):            /home/primarina314/opencv_contrib/modules
    Version control (extra):     4.11.0-42-gacd145f4

  Platform:
    Timestamp:                   2025-04-28T09:41:11Z
    Host:                        Linux 5.15.167.4-microsoft-standard-WSL2 x86_64
    CMake:                       3.31.2
    CMake ge

2025-05-09 01:20:50.722870: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-09 01:20:50.723071: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-09 01:20:50.723099: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-09 01:20:50.723509: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-09 01:20:50.723583: I tensorflow/core/common_runtime/gpu/gpu

In [1]:
# from laptop w/o gpu
import numpy as np
import cv2
import time
import os
from enum import Enum
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
from utils import *

print(cv2.__version__)

%load_ext autoreload
%autoreload 2

4.12.0-dev


# 경계 블렌딩

우선 아래 방법 적용하지 않은 상태로 구현

- 원본 파트에서 경계에서 떨어진 거리에 따라 블렌딩 비율 다르게
- 경계에 가까울수록 downscaled->upscaled 이미지 비율을 높게 잡고
- 경계에서 멀어질수록 original part 이미지 비율을 높게 설정하여 블렌딩
- 일정 거리 이상으로 멀어지면 100% 원본파트
- 선형, sigmoid 등 함수로 interpolation
- roi 선택 시에 각 좌표를 모두 저장 -> 좌표를 바탕으로 거리 계산 -> 곡선형태로 저장할 수도 있으므로 더 광범위하게 적용가능한 findContour 함수로 처리하는게 나을듯

In [2]:
class blend_mode(Enum):
    alpha = 0
    poisson = 1
    laplacian = 2
    grad = 3
    dl = 4

def blending_two_images(src, target, mask, mode = blend_mode.poisson, center = (0, 0)):
    match mode:
        case blend_mode.alpha:
            result = cv2.addWeighted(src, 0.7, target, 0.3, 0, mask=mask)
        case blend_mode.poisson:
            result = cv2.seamlessClone(
                src, target, mask, center, cv2.MIXED_CLONE
            )
        case blend_mode.laplacian:
            # laplacian dist
            return
        case blend_mode.grad:
            # gradient
            return
        case blend_mode.dl:
            # dl model
            return

    return result

In [12]:
img_path = 'Lenna_(test_image).png'

# down->upscaled 된거 path
# original part path

output_path = 'blending-test'

# 블렌딩

In [66]:
from utils import MRS3
roi, _ = MRS3.select_polygon_roi(img_path)
mask = np.all(roi != [0, 0, 0], axis=2)
mask_uint8 = mask.astype(np.uint8) * 255

In [67]:
cv2.imwrite('bin-img.png', mask_uint8, [cv2.IMWRITE_PNG_COMPRESSION, 9, cv2.IMWRITE_PNG_BILEVEL, 1])
cv2.imwrite('bin-img.jpeg', mask_uint8)
cv2.imwrite('roi-test.png', roi, [cv2.IMWRITE_PNG_COMPRESSION, 9])

True

In [68]:
print(os.path.getsize('bin-img.png'))
print(os.path.getsize('roi-test.png'))

print(os.path.getsize('Lenna_(test_image).png'))

758
125725
473831


In [15]:
cv2.imshow('test', mask_uint8)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:


def point_line_distance(point, line_start, line_end):
    """
    점과 선분 사이의 최소 수직거리 계산
    point: (x, y) 튜플
    line_start, line_end: 선분의 두 끝점 (x, y) 튜플
    """
    px, py = point
    x1, y1 = line_start
    x2, y2 = line_end

    # 선분 벡터
    line_vec = np.array([x2 - x1, y2 - y1])
    # 점과 선분 시작점 벡터
    point_vec = np.array([px - x1, py - y1])

    line_len_sq = np.dot(line_vec, line_vec)
    if line_len_sq == 0:
        # 선분이 점인 경우
        return np.linalg.norm(point_vec)

    # 점에서 선분으로의 투영 비율
    t = np.dot(point_vec, line_vec) / line_len_sq

    if t < 0.0:
        # 투영점이 선분 시작점 밖에 있는 경우
        closest_point = np.array([x1, y1])
    elif t > 1.0:
        # 투영점이 선분 끝점 밖에 있는 경우
        closest_point = np.array([x2, y2])
    else:
        # 투영점이 선분 위에 있는 경우
        closest_point = np.array([x1, y1]) + t * line_vec

    # 점과 가장 가까운 점 사이 거리
    dist = np.linalg.norm(np.array([px, py]) - closest_point)
    return dist

def is_point_inside_polygon(point, polygon):
    """
    점이 다각형 내부에 있는지 확인 (Ray casting algorithm)
    point: (x, y) 튜플
    polygon: 다각형 꼭짓점 좌표 배열 [(x1,y1), (x2,y2), ...]
    """
    x, y = point
    n = len(polygon)
    inside = False
    
    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    
    return inside

# class 


def distance_to_polygon_edge_from_mask(mask, point):
    """
    mask: 2D numpy array, 다각형 영역이 1(또는 255), 배경이 0인 바이너리 마스크
    point: (x, y) 튜플, 기준점 좌표

    반환값: point에서 다각형 경계까지의 최소 수직거리 (float)
    """
    # 마스크를 바이너리(0 또는 255)로 변환
    mask_bin = (mask > 0).astype(np.uint8) * 255

    # 다각형 외곽선(경계) 검출
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None  # 다각형이 없는 경우
    
    # 다각형 꼭짓점 좌표 추출
    polygon = contours[0].reshape(-1, 2)

    # 점이 다각형 내부에 있는지 확인
    inside = is_point_inside_polygon(point, polygon)

    min_dist = float('inf')
    for contour in contours:
        pts = contour.reshape(-1, 2)
        n = len(pts)
        for i in range(n):
            start = tuple(pts[i])
            end = tuple(pts[(i + 1) % n])
            dist = point_line_distance(point, start, end)
            if dist < min_dist:
                min_dist = dist

    return min_dist if inside else -min_dist

def distance_to_polygon_edge_from_contours(contours, point):
    """
    point: (x, y) 튜플, 기준점 좌표
    contours: contours 튜플(vertices). cv2.findContours 의 0번째 컴포넌트
    반환값: point에서 다각형 경계까지의 최소 수직거리 (float) - 부호 고려
    """

    if len(contours) == 0:
        return None  # 다각형이 없는 경우
    
    # 다각형 꼭짓점 좌표 추출
    polygon = contours[0].reshape(-1, 2)

    # 점이 다각형 내부에 있는지 확인
    inside = is_point_inside_polygon(point, polygon)

    min_dist = float('inf')
    for contour in contours:
        pts = contour.reshape(-1, 2)
        n = len(pts)
        for i in range(n):
            start = tuple(pts[i])
            end = tuple(pts[(i + 1) % n])
            dist = point_line_distance(point, start, end)
            if dist < min_dist:
                min_dist = dist
    return min_dist if inside else -min_dist

# 사용 예시
# mask = np.zeros((200, 200), dtype=np.uint8)
# polygon = np.array([[50, 50], [150, 50], [150, 150], [50, 150]])
# cv2.fillPoly(mask, [polygon], 255)

# point = (100, 100)  # 예시 기준점
# distance = distance_to_polygon_edge(mask, point)
# print(f"Distance from point {point} to polygon edge: {distance}")


In [8]:
# dist = distance_to_polygon_edge(mask, (100, 100))
# print(dist)
mask_bin = (mask > 0).astype(np.uint8) * 255

contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

def check_dist(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # dist = distance_to_polygon_edge_from_mask(mask, (x, y))
        dist = distance_to_polygon_edge_from_contours(contours, (x, y))
        print(f"distance from ({x}, {y}) to edge: {dist}")

cv2.namedWindow("test")
cv2.setMouseCallback("test", check_dist)

cv2.imshow('test', mask_uint8)

clicking = True
while True:
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

distance from (73, 92) to edge: 43.04648650006177
distance from (40, 22) to edge: -33.54101966249684
distance from (64, 49) to edge: 1.0
distance from (42, 51) to edge: -7.211102550927978
distance from (42, 55) to edge: -3.605551275463989
distance from (42, 57) to edge: -2.0
distance from (39, 65) to edge: 3.605551275463989
distance from (34, 64) to edge: 0.7071067811865476
distance from (69, 107) to edge: 54.56189146281496
distance from (3, 29) to edge: -44.721359549995796
distance from (85, 124) to edge: 38.01315561749642
distance from (29, 31) to edge: -30.870698080866262
distance from (79, 110) to edge: 47.853944456021594
distance from (72, 90) to edge: 40.8166632639171
distance from (90, 77) to edge: 34.713109915419565


In [ ]:
mrs3_compress(img_path, output_path, 4, ROI_mode.Polygon)


original file: 473831
downscaled filesize: 35816
roi filesize: 27504
config filesize: 97
compression ratio: 0.1338388581582885


In [6]:
mrs3_restore('qwergfhfghgfhgfg', MRS3_mode.edsr, 'qwergfhfghgfhgfg/restored.png')

In [7]:
def distance_to_polygon_edge(mask, point):
    """
    mask: 2D numpy array, polygon 영역이 1(또는 255), 배경이 0인 바이너리 마스크
    point: (x, y) 튜플, 기준점 좌표

    반환값: point에서 다각형 경계까지의 최단 거리 (float)
    """
    # 마스크를 바이너리(0 또는 255)로 변환
    mask_bin = (mask > 0).astype(np.uint8) * 255

    # 다각형 외곽선(경계) 검출
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None  # 다각형이 없는 경우

    # 모든 경계점에 대해 거리 계산, 최솟값 반환
    min_dist = float('inf')
    for contour in contours:
        for pt in contour:
            dist = np.linalg.norm(np.array(pt[0]) - np.array(point))
            if dist < min_dist:
                min_dist = dist

    return min_dist

# 사용 예시
# mask: 다각형이 그려진 바이너리 마스크 (예: 200x200, 가운데 사각형)
# mask = np.zeros((200, 200), dtype=np.uint8)
# polygon = np.array([[50, 50], [150, 50], [150, 150], [50, 150]])
# cv2.fillPoly(mask, [polygon], 255)

# point = (100, 100)  # 예시 기준점

# distance = distance_to_polygon_edge(mask, point)
# print(f"Distance from point {point} to polygon edge: {distance}")


In [ ]:
# TODO: 경계 interpolation 한계 설정
# 거리에 따라 자연스럽게


In [ ]:
class blending_interpolation_mode(Enum):
    LINEAR = 0
    HERMIT_3 = 1
    HERMIT_5 = 2
    SINUSOIDAL = 3
    EXPONENTIAL = 4
    SINH = 5

# replace s = (x - a) / (b - a)
# f(x) = (x - a) / (b - a)
# f(x) = 3x² - 2x³
# f(x) = 6x⁵ - 15x⁴ + 10x³
# f(x) = (1 - cos(πx)) / 2



def get_alpha():

    return

def blending_with_interpolation():

    return

In [72]:
class Person:
    def __init__(self, first_name, last_name, age):
        self.first_name = first_name
        self.last_name = last_name
        self.__age = age  # 내부 변수로 저장
        
    @property
    def age(self):
        return self.__age
    
    @age.setter
    def age(self, newage):
        self.__age = newage


p = Person("qwer", "asdf", 20)

In [73]:
print(p.age)

20


In [78]:
print(type('asdf'))

<class 'str'>


In [6]:
import utils as ut

In [ ]:
import mrs3 as mr



asdfasdfasdfasd
